# Assignment 2

#### First Name: David
#### Last Name: Rosado Rodríguez


## 1. Load Data from JSON

In [103]:

from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Twitter Analysis")\
.getOrCreate()


In [104]:
df_twitter = spark.read.json("corona_tweet_new.json")

In [105]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- frie

In [106]:
### From the user nestec col select the following cols only id_str,followers_count,friends_count and created at 
# (2 points)
from pyspark.sql.functions import col 
# To implement this exercise, we will us the function select() to select the columns we want

df_twitter=df_twitter.select(col("created_at"),
                             col("favorite_count"),
                             col("hashtags"),
                             col("id"),
                             col("in_reply_to_status_id"),
                             col("in_reply_to_user_id_str"),
                             col("location"),
                             col("reply_count"),
                             col("retweet_count"),
                             col("source"),
                             col("user.id_str"),
                             col("user.followers_count"),
                             col("user.friends_count"),
                             col("user.created_at").alias("user_created_at") # to avoid colisions
                            )

In [107]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- id_str: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)



In [108]:
# Print the total count of number of records in df_twitter(1 point)
df_twitter.count()

15894

In [109]:
# Extract the source lable from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regular expresion and the group would be 1 for this regular expression.
#(4 points)

from pyspark.sql.functions import regexp_extract, col

# For this exercise we use the function withColumn() to add a new column called extracted_source
# We also use the function regexp_extract() to extract the label from the source column using the regular expression provided

df_twitter = df_twitter.withColumn("extracted_source", regexp_extract(col("source"), '<a [^>]+>([^<]+)', 1))
df_twitter.select(col('extracted_source'),col('source')).show()

+-------------------+--------------------+
|   extracted_source|              source|
+-------------------+--------------------+
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter Web Client|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
+----------

In [110]:
# Convert the DataFrame into RDD
rdd_twitter=df_twitter.rdd.map(tuple)

In [111]:
# Create a temporay table in memory with name as twitter (1 point)
# For this exercise we can use the function createOrReplaceTempView()
# This method creates or replaces a temporary view in the Spark catalog
df_twitter.createOrReplaceTempView("twitter")

## 2. Analyze Data

#### You will be writing code to find the answer to the questions listed below using Just RDD, Using spark SQL 

- Analyze using RDD 
- Analyze using Dataframe without temp table 
- Analyze using spark.sql with temp table


#### 2.1 Get total number of unique users (1 point for each type)

In [112]:
# Using RDD
# tweet[10]-->id_str

unique_users = rdd_twitter.map(lambda tweet: (tweet[10], 1)).reduceByKey(lambda a, b: a + b).count()

print('Total number of unique users:',unique_users)

Total number of unique users: 14094


In [113]:
# Using DataFrame
unique_users = df_twitter.select("id_str").distinct().count()

print('Total number of unique users:',unique_users)

Total number of unique users: 14094


In [114]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(DISTINCT id_str) FROM twitter').show()

+----------------------+
|count(DISTINCT id_str)|
+----------------------+
|                 14094|
+----------------------+



#### 2.2 Get count of user who have more than 1 tweet in the data (2 points)

In [115]:
# Using RDD
more_than_one = rdd_twitter.map(lambda tweet: (tweet[10], 1)).reduceByKey(lambda a, b:
                                                          a + b).filter(lambda user: user[1] >= 2).count()

print('Number of users with more than 1 tweet in the data:',more_than_one)

Number of users with more than 1 tweet in the data: 1016


In [116]:
# Using DataFrame
more_than_one = df_twitter.groupby("id_str").count().filter("count>=2").count()

print('Number of users with more than 1 tweet in the data:',more_than_one)

Number of users with more than 1 tweet in the data: 1016


In [117]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(*) FROM (SELECT COUNT(id_str) AS tweet_count FROM twitter GROUP BY id_str) WHERE tweet_count >= 2').show()

+--------+
|count(1)|
+--------+
|    1016|
+--------+



#### 2.3 Get total number unique extracted_source (1 point each)

In [118]:
# Using RDD
# tweet[14]-->extracted_source

unique_extr = rdd_twitter.map(lambda tweet: (tweet[14], 1)).reduceByKey(lambda a, b: a + b).count()

print('The total number of unique extracted_source:',unique_extr)

The total number of unique extracted_source: 133


In [119]:
# Using DataFrame
unique_extr = df_twitter.select("extracted_source").distinct().count()

print('The total number of unique extracted_source:',unique_extr)

The total number of unique extracted_source: 133


In [120]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(DISTINCT extracted_source) FROM twitter').show()

+--------------------------------+
|count(DISTINCT extracted_source)|
+--------------------------------+
|                             133|
+--------------------------------+



#### 2.4 Get top 5 most used extracted_source

In [121]:
# Using RDD (5 points)
rdd_twitter.map(lambda tweet: (tweet[14], 1)).reduceByKey(lambda a, b: a + b).takeOrdered(5, lambda s: -1 * s[1])

[('Twitter for Android', 6262),
 ('Twitter for iPhone', 5698),
 ('Twitter Web App', 2878),
 ('Twitter for iPad', 428),
 ('Twitter Web Client', 136)]

In [122]:
# Using DataFrame (2 points)
df_twitter.groupby("extracted_source").count().sort("count", ascending = False).show(5)

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+
only showing top 5 rows



In [123]:
# Using spark.sql and the temporay table. (2 points)
spark.sql('SELECT extracted_source, COUNT(extracted_source) as count FROM twitter GROUP BY extracted_source ORDER BY count DESC LIMIT 5').show()

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+



#### 2.5 Get count of distinct hastags used ( 5 point each) 

Careful with the structure of the hashtags column. We need to use `pyspark.sql.functions.explode()` (using the dataframe) to transform a column of arrays or maps into multiple rows, with each row containing one element of the array or one key-value pair of the map.

Example:

 +---+---------+\
 | id|   values|\
 +---+---------+\
 |  1|(1, 2, 3)|\
 |  2|   (4, 5)|\
 |  3|      (6)|\
 +---+---------+\
 
to 

 +---+-----+\
 | id|value|\
 +---+-----+\
 |  1|    1|\
 |  1|    2|\
 |  1|    3|\
 |  2|    4|\
 |  2|    5|\
 |  3|    6|\
 +---+-----+\
 
The equivalent in rdd is `flatMap`.

In [124]:
# Using RDD
# tweet[2]-->hashtags
distinct_hasht = rdd_twitter.flatMap(lambda tweet: tweet[2]).map(lambda hashtags: (hashtags, 1)).reduceByKey(lambda a, b: a + b).count()

print('The number of distinct hastags used is:',distinct_hasht)

The number of distinct hastags used is: 1215


In [125]:
# Using DataFrame
from pyspark.sql.functions import explode

distinct_hasht  = df_twitter.select(explode(col('hashtags'))).distinct().count()
print('The number of distinct hastags used is:',distinct_hasht)

The number of distinct hastags used is: 1215


In [126]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(*) FROM (SELECT DISTINCT EXPLODE(hashtags) FROM twitter)').show()

+--------+
|count(1)|
+--------+
|    1215|
+--------+



#### 2.6 Get top 5 hashtags

In [127]:
# Using RDD (4 points)
rdd_twitter.flatMap(lambda tweet: tweet[2]).map(lambda hashtags: 
                                                (hashtags, 1)).reduceByKey(lambda a, b:a + b).takeOrdered(5, lambda s: -1 * s[1])

[('طبق_القدرات_للثانويه_ياريس', 385),
 ('Corona', 319),
 ('OilPrice', 251),
 ('COVID19', 125),
 ('corona', 123)]

In [128]:
# Using DataFrame (2 points)
df_twitter.select(explode(col('hashtags'))).groupby("col").count().sort("count", ascending = False).show(5)

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+
only showing top 5 rows



In [129]:
# Using spark.sql and the temporay table. (2 points)
spark.sql('SELECT col, COUNT(col) as count FROM (SELECT EXPLODE(hashtags) FROM twitter) GROUP BY col ORDER BY count DESC LIMIT 5').show()

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+



#### 2.6 Get total number of tweets which are retweeted more than 100 times

In [130]:
# Using RDD
# tweet[8]--> retweet_count
morethan100 = rdd_twitter.filter(lambda tweet: tweet[8] > 100).count()

print('The number of tweets which are retweeted more than 100 times is:',morethan100)

The number of tweets which are retweeted more than 100 times is: 15753


In [131]:
# Using DataFrame

morethan100 = df_twitter.filter("retweet_count > 100").count()

print('The number of tweets which are retweeted more than 100 times is:',morethan100)

The number of tweets which are retweeted more than 100 times is: 15753


In [132]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(id) FROM twitter WHERE retweet_count > 100').show()

+---------+
|count(id)|
+---------+
|    15753|
+---------+



#### 2.8 Get top 3 most retweeted tweets per country (8 points)

In [133]:
# Using RDD
from operator import itemgetter
# tweet[6]-->location
# tweet[3]-->id
# tweet[8]-->retweet_count
rdd_twitter.map(lambda tweet: (tweet[6], [tweet[3], tweet[8]])).groupByKey().mapValues(lambda x: sorted(list(x), key=itemgetter(1), reverse=True)[:3]).collect()

[('USA',
  [['1252331777806524416', 9994],
   ['1252254239805579264', 9987],
   ['1252335464750735362', 9982]]),
 ('Pakistan',
  [['1252334264248606720', 9988],
   ['1252251912084357121', 9975],
   ['1252252126694309888', 9973]]),
 ('Mexico',
  [['1252253843145912320', 9998],
   ['1252255209776189442', 9994],
   ['1252252016006422533', 9971]]),
 ('Spain',
  [['1252335445876367361', 9992],
   ['1252334839094599681', 9981],
   ['1252254696112300032', 9969]]),
 ('Canada',
  [['1252335430323888128', 9997],
   ['1252254877939531776', 9992],
   ['1252252082825986051', 9987]]),
 ('Germany',
  [['1252334028092399622', 9999],
   ['1252330902325248000', 9997],
   ['1252252295510855682', 9990]]),
 ('India',
  [['1252332114948874240', 9988],
   ['1252252336921206787', 9976],
   ['1252254519116746754', 9973]]),
 ('China',
  [['1252335780707684352', 9998],
   ['1252253596516843520', 9993],
   ['1252255562525560832', 9984]]),
 ('UK',
  [['1252333018578145280', 9991],
   ['1252252091822870529', 9989],

In [135]:
# Using DataFrame
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

df_twitter.select('id','location','retweet_count', rank().over(Window().partitionBy("location").orderBy(col("retweet_count").desc())).alias('rank')).sort("retweet_count",
    ascending=False).filter('rank <= 3').sort("location", "rank").show()

+-------------------+--------+-------------+----+
|                 id|location|retweet_count|rank|
+-------------------+--------+-------------+----+
|1252335430323888128|  Canada|         9997|   1|
|1252254877939531776|  Canada|         9992|   2|
|1252252082825986051|  Canada|         9987|   3|
|1252253612140490759|   Chile|         9988|   1|
|1252334891951427585|   Chile|         9984|   2|
|1252253710182481920|   Chile|         9978|   3|
|1252335780707684352|   China|         9998|   1|
|1252253596516843520|   China|         9993|   2|
|1252255562525560832|   China|         9984|   3|
|1252334028092399622| Germany|         9999|   1|
|1252330902325248000| Germany|         9997|   2|
|1252252295510855682| Germany|         9990|   3|
|1252332114948874240|   India|         9988|   1|
|1252252336921206787|   India|         9976|   2|
|1252254519116746754|   India|         9973|   3|
|1252252106750377996|   Italy|         9994|   1|
|1252251206027816960|   Italy|         9984|   2|


In [136]:
# Using spark.sql and the temporay table.
spark.sql('SELECT id, location, retweet_count FROM (SELECT *, RANK() OVER (PARTITION BY location ORDER BY retweet_count DESC) as rank FROM twitter) WHERE rank <= 3 ORDER BY location, rank ASC').show()

+-------------------+--------+-------------+
|                 id|location|retweet_count|
+-------------------+--------+-------------+
|1252335430323888128|  Canada|         9997|
|1252254877939531776|  Canada|         9992|
|1252252082825986051|  Canada|         9987|
|1252253612140490759|   Chile|         9988|
|1252334891951427585|   Chile|         9984|
|1252253710182481920|   Chile|         9978|
|1252335780707684352|   China|         9998|
|1252253596516843520|   China|         9993|
|1252255562525560832|   China|         9984|
|1252334028092399622| Germany|         9999|
|1252330902325248000| Germany|         9997|
|1252252295510855682| Germany|         9990|
|1252332114948874240|   India|         9988|
|1252252336921206787|   India|         9976|
|1252254519116746754|   India|         9973|
|1252252106750377996|   Italy|         9994|
|1252251206027816960|   Italy|         9984|
|1252330500670337024|   Italy|         9971|
|1252253843145912320|  Mexico|         9998|
|125225520

#### 2.9 Total number of tweets per country

In [137]:
# Using RDD (3 points)
# tweet[6]-->location
rdd_twitter.map(lambda tweet: (tweet[6], 1)).reduceByKey(lambda a, b: a + b).collect()

[('USA', 1539),
 ('Pakistan', 1470),
 ('Mexico', 1409),
 ('Spain', 1464),
 ('Canada', 1441),
 ('Germany', 1426),
 ('India', 1480),
 ('China', 1457),
 ('UK', 1376),
 ('Chile', 1410),
 ('Italy', 1422)]

In [138]:
# Using DataFrame (2 points)
df_twitter.groupBy("location").count().show()

+--------+-----+
|location|count|
+--------+-----+
| Germany| 1426|
|   India| 1480|
|   China| 1457|
|   Chile| 1410|
|   Italy| 1422|
|   Spain| 1464|
|     USA| 1539|
|  Mexico| 1409|
|      UK| 1376|
|  Canada| 1441|
|Pakistan| 1470|
+--------+-----+



In [139]:
# Using spark.sql and the temporay table. (1 point)
spark.sql('SELECT location, COUNT(id) FROM twitter GROUP BY location').show()

+--------+---------+
|location|count(id)|
+--------+---------+
| Germany|     1426|
|   India|     1480|
|   China|     1457|
|   Chile|     1410|
|   Italy|     1422|
|   Spain|     1464|
|     USA|     1539|
|  Mexico|     1409|
|      UK|     1376|
|  Canada|     1441|
|Pakistan|     1470|
+--------+---------+



## 3. Save Data

#### 3.1 save the data such that you have seperate folder per country (2 points)

In [140]:
# Using DataFrame
# We need first to convert the hashtags column.
# Column `hashtags` has a data type of array<string>, which is not supported by CSV.
df_twitter = df_twitter.withColumn("hashtags", col("hashtags").cast("string"))
df_twitter.write.partitionBy('location').csv("location", header = 'true')

#### 3.2 Save the data as parquet files (1 points)

In [141]:
# Using DataFrame
df_twitter.write.partitionBy('location').parquet("parquet_location")